# Pokemon Battle (fino a 6 pt)
Questo homework sarà un po' diverso dagli altri, perchè non voglio darvi una traccia già definita e/o una soluzione unica perchè voglio che sviluppiate questo progetto con le vostre idee nelle parti che vi interessano di più (o forse sono solo pigro).
L'obiettivo è ampliare e aggiungere funzionalità al progetto della battaglia Pokemon vista a lezione e vi propongo alcune vie che secondo me ha senso percorrere, ma apprezzerò anche idee innovative!
Uniche regole da seguire:
- no programmazione grafica, voglio solo codice a terminale
- nella soluzione scrivete un breve riassunto delle nuove funzionalità che avete aggiunto
- fate quante proposte volete, il punteggio dipenderà sia da qualità che quantità delle vostre aggiunte
- potete modificare il mio codice come volete (e vi suggerisco di farlo), ma cercate di mantenere i meccanismi base
- scrivete codice "bello" (sfruttate le vostre nuove conoscenze!) e leggibile
- assicuratevi che tutto funzioni quando consegnate!

Come potrete vedere, le proposte sono molto sinergiche tra di loro e sicuramente farne mole porta ad un risultato sempre più soddisfacente.
In particolare la 1 e la 2 credo siano le più semplici e sono in grado di dare più valore alle altre se sviluppate con loro.

Se volete la lode sorprendetemi!

## Proposta 1: Creazione Allenatore (1pt)
Create una classe Allenatore tale che abbia una squadra e 'spostate' le funzioni per fare partire una lotta all'interno dell'allenatore. Considerate di potere fare scontri sia contro altri allenatori che pokemon selvaggi (senza allenatore), ed implementate la scelta di cambiare pokemon in una lotta e di fuggire da un pokemon selvaggio.
Infine create una sorta di inventario con pozioni per la salute e pokeball da usare contro i pokemon selvaggi, ovviamente aggiungete anche la possibilità di usare oggetti nelle lotte.

## Proposta 2: Completare la Prima Generazione (1pt)
Create un file con la generazione dei singoli Pokemon del Pokedex della prima generazione e fate in modo che quando volete crearlo nel vostro codice vi basterà digitare il nome ed i dati verranno presi dal file.
Aggiungete anche tutte le immagini dei Pokemon nel file già presente. Per fare questo lavoro dovrete fare un altro script (consegnate anche quello) che vi consenta di riscrivere sul nostro file nel formato che volete i disegni presenti nel sito:
https://gist.github.com/MatheusFaria/4cbb8b6dbe33fd5605cf8b8f7130ba6d

## Proposta 3: Miglioramento Statistiche (2pt)
Create le seguenti statistiche all'interno dei Pokemon: LVL, HP, ATT, DEF, SPD, SPC, TIPO1, TIPO2
Il livello è un numero da 1 a 100 ed influenzerà le 5 stat successive, infatti queste hanno un valore base al lvl 1, diverso per ogni Pokemon ed al lvl 100 gli HP sono il doppio +110 mentre le altre sono il doppio +5; create un incremento lineare sapendo questa informazione e prendete le statistiche base (lvl 1) da qui: https://bulbapedia.bulbagarden.net/wiki/List_of_Pok%C3%A9mon_by_base_stats_(Generation_I).
Sistematizzate meglio la gestione dei tipi dei Pokemon e create la tavola che spiega i modificatori di attacco tra i tipi, potete trovarla qui: https://pokemondb.net/type/old (GEN 1)
Infine fate in modo che il Pokemon più veloce attacchi per primo in ogni turno e cambiate la formula dei danni con:

**( ( (LVL * 2/5 +2 ) * ATT * POW )/( 50*DEF ) +2 ) * STAB * TYPEMOD * RANDOM)/255**

POW: Potenza Attacco (se non fate la prossima proposta settate 50 e tutto fisico)
STAB: Vale 1.5 se l'attacco è del tipo del Pokemon che attacca, altrimenti 1
TYPEMOD: Prodotto dei moltiplicatori di tipo (tipo dell'attacco contro entrambi i tipi del difensore)
RANDOM: Numero tra 217 e 255

## Proposta 4: Miglioramento Attacchi (2pt)
Create un file contenete le statistiche di ogni mossa Pokemon, scegliete pure solo gli attacchi e le modifiche di status semplici. Ogni attacco deve avere PP (quante volte si può usare la mossa), POW (forza), ACC (probabilità di riuscita), TIPO e CAT (fisico, speciale o status). Vi ricordo che una mossa speciale coinvolge sia in attacco che in difesa il valore SPC al posto di ATT e DEF, mentre una mossa di status non fa danni.
Create anche i modificatori di stato come ad esempio addormentato, avvelenato, confuso...
Infine sorprendetemi con qualche meccanica complessa!

## Codice Sviluppato a Lezione:



In [34]:
import sys
import random
import time

def delay_print(s):
    for c in s:
        sys.stdout.write(c)
        sys.stdout.flush()
        time.sleep(0.03)

In [35]:
from collections import namedtuple

Stats = namedtuple('Stats', 'hp attack defense sp_atk sp_def speed')

class EV(object):
    def __init__(self, hp, attack, defense, sp, speed):
        self.hp = hp
        self.attack = attack
        self.defense = defense
        self.sp = sp
        self.speed = speed

class Pokemon(object):
    _base_stats = Stats(0, 0, 0, 0, 0, 0)
    attackingDict = {'fire': {'fire': 0.5, 'grass': 2.0, 'water': 0.5, 'normal': 1.0, 'flying':1.0, 'bug':2.0, 'poison':1.0, 'electric': 1.0}, 
                     'grass': {'fire': 0.5, 'grass': 0.5, 'water': 2.0, 'normal': 1.0, 'flying':1.0, 'bug':0.5, 'poison':0.5, 'electric': 1.0}, 
                     'water': {'fire': 2.0, 'grass': 0.5, 'water': 0.5, 'normal': 1.0, 'flying':1.0, 'bug':1.0, 'poison':1.0, 'electric': 1.0},
                    'normal': {'fire': 1.0, 'grass': 1.0, 'water': 1.0, 'normal': 1.0, 'flying':1.0, 'bug':1.0, 'poison':1.0, 'electric': 1.0},
                     'flying': {'fire': 1.0, 'grass': 1.0, 'water': 1.0, 'normal': 1.0, 'flying':1.0, 'bug':2.0, 'poison':1.0, 'electric': 1.0}, 
                     'bug': {'fire': 0.5, 'grass': 2.0, 'water': 1.0, 'normal': 1.0, 'flying':0.5, 'bug':1.0, 'poison':2.0, 'electric': 1.0},
                    'poison': {'fire': 1.0, 'grass': 1.0, 'water': 1.0, 'normal': 1.0, 'flying':1.0, 'bug':2.0, 'poison':1.0, 'electric': 1.0},
                    'electric': {'fire': 1.0, 'grass': 0.5, 'water': 2.0, 'normal': 1.0, 'flying':2.0, 'bug':1.0, 'poison':2.0, 'electric': 0.5}}
    def __init__(self, name, type, attacks, typeattack, LVL, HP, ATT, DEF, SPD, SPC, type2):
        self.name = name
        self.HP = HP      
        self.ATT = ATT
        self.type = type
        self.attacks = attacks
        self.typeattack = typeattack
        self.LVL = LVL
        self.DEF = DEF
        self.SPD = SPD
        self.SPC = SPC
        self.type2 = type
        self.max_health = LVL * 5
        self.health = LVL * 5
        self.experience = 0
## Ho cercato la formula per calcolare Evs, sperando che funzioni    
    @property
    def max_hp(self):
        base = self._base_stats.hp
        iv = self._iv.hp
        ev = self._ev.hp
        level = self.level
        return int(((2 * base + iv + int(ev/4)) * level) / 100) + level + 10

    def _calculate_stat(self, stat):
        base = getattr(self._base_stats, stat)
        iv = getattr(self._iv, stat)
        ev = getattr(self._ev, stat)
        level = self.level
        # Ci sarebbe da inserire anche la natura del pokemon
        nature = 1
        return int((int(((2 * base + iv + int(ev/4)) * level) / 100) + 5) * nature)
        
        
    def __str__(self):
        fileName = "PokemonArt"
        image = R""" """
        image += f"""{self.name} - {self.HP}HP\n"""
        file = open(f"{fileName}.txt", "r")
        text = file.readlines()
        add = False
        for line in text:
            if(line.strip() == self.name):
                add = True
            elif(add):
                if(line.strip() == "STOP"):
                    return image
                else:
                    image += line.rstrip()
                    image += """\n"""
        return "Immagine non Trovata"
        
        
    def startBattle(self, Opponent, humanPlayer, pokemonArt):
        if(pokemonArt):
            delay_print("--- BATTAGLIA POKEMON ---\n")
            print(self)
            delay_print("--- VERSUS ---\n")
            print(Opponent)
        else:
            delay_print("--- BATTAGLIA POKEMON ---")
            delay_print(f"\n{self.name} - {self.HP}HP\t VS\t {Opponent.name} - {self.HP}HP\n")
        return self.battle(Opponent, True, humanPlayer)
    
    
    def attackChoice(self, Opponent, you, humanPlayer):
        if(you and humanPlayer):
            for i, x in enumerate(self.attacks):
                print(f"{i+1}.", x)
            attackStr = input('Scegli una mossa: ')
            while(attackStr!="1" and attackStr!="2" and attackStr!="3" and attackStr!="4"):
                attackStr = input('Scegli una mossa valida: ')
            return int(attackStr)-1
        else:
            return random.randint(0,3)
        
        
    def attack(self, Opponent, attackCode):
        attackName = self.attacks[attackCode]
        
        attackMod = 1
        if(self.typeattack[attackCode]):
            attackMod = self.attackingDict[self.type][Opponent.type]
        attackDamage = self.ATT * attackMod
        
        strDamage = ""
        if(attackMod < 1):
            strDamage += "\nNon è molto efficace..."
        elif(attackMod > 1):
            strDamage += "\nE' superefficace!"
            
        return attackName, attackDamage, strDamage
        def opp_adv(self, opponent):
            damage_amt = round(self.level * 2)
            print('{name} attacked {opp_name} for {damage} damage points. It was super effective!'.format(name = self.name, opp_name = opponent.name, damage = damage_amt))
            opponent.lose_health(damage_amt)
            self.experience += 5
            if self.experience >= 20:
                self.level += 1
                self.experience = 0
                print('{name} leveled up!'.format(name = self.name))
        def opp_dis(self, opponent):
            damage_amt = round(self.level * 0.5)
            print('{name} attacked {opp_name} for {damage} damage points. It wasn\'t very effective.'.format(name = self.name, opp_name = opponent.name, damage = damage_amt))
            opponent.lose_health(damage_amt)
            self.experience += 2
            if self.experience >= 20:
                self.level += 1
                self.experience = 0
                print('{name} leveled up!'.format(name = self.name))
    
    
    def battle(self, Opponent, you, humanPlayer):   
        
        if(self.HP > 0):
            attackCode = self.attackChoice(Opponent, you, humanPlayer)
            attackName, attackDamage, strDamage = self.attack(Opponent, attackCode)
            Opponent.HP -= attackDamage 
            
            delay_print(f"\n{self.name} usa {attackName}")
            delay_print(strDamage)
            delay_print(f"\n{Opponent.name} ha {max(Opponent.HP,0)} HP rimasti\n")
            
            return Opponent.battle(self, not(you), humanPlayer)
        
        else:
            money = random.randint(100,5000)
            delay_print(f"\n{Opponent.name} vince! ({Opponent.HP} HP rimasti)")
            """if(you): delay_print(f"\nDai all'avversario ${money}.\n")
            else: delay_print(f"\nL'avversario ti da' ${money}.\n")"""
            return Opponent, self
    

In [36]:
# self, name, type, attacks, typeattack, LVL, HP, ATT, DEF, SPD, SPC, type2
Bulbasaur  = Pokemon('Bulbasaur' , 'grass', ['Frustata', 'Testata', 'Azione', 'Semitraglia']      , [1,0,0,1], 5, 45, 49, 49, 45, 65, 'grass')
Ivysaur    = Pokemon('Ivysaur'   , 'grass', ['Frustata', 'Foglielama', 'Semitraglia', 'Azione']   , [1,1,1,0], 25, 60, 62, 63, 60, 80, 'grass' )
Venusaur   = Pokemon('Venusaur'  , 'grass', ['Frustata', 'Foglielama', 'Terremoto', 'Radicalbero'], [1,1,0,1], 50, 80, 82, 83, 80, 100, 'poison')

Charmander = Pokemon('Charmander', 'fire' , ['Braciere', 'Graffio', 'Azione', 'Fuocopugno']       , [1,0,0,1], 5, 45, 49, 49, 45, 65, 'fire')
Charmeleon = Pokemon('Charmeleon', 'fire' , ['Braciere', 'Graffio', 'Lanciafiamme', 'Fuocopugno'] , [1,0,1,1], 25, 60, 62, 63, 60, 80, 'fire')
Charizard  = Pokemon('Charizard' , 'fire' , ['Lanciafiamme', 'Volo', 'Incendio', 'Fuocopugno']    , [1,0,1,1], 50, 80, 82, 83, 80, 100, 'flying')

Squirtle   = Pokemon('Squirtle'  , 'water', ['Bollaraggio', 'Azione', 'Testata', 'Surf']          , [1,0,0,1], 5, 45, 49, 49, 45, 65, 'water')
Wartortle  = Pokemon('Wartortle' , 'water', ['Bollaraggio', 'Pistolacqua', 'Testata', 'Surf']     , [1,1,0,1], 25, 60, 62, 63, 60, 80, 'water')
Blastoise  = Pokemon('Blastoise' , 'water', ['Bollaraggio', 'Pistolacqua', 'Idropompa', 'Surf']   , [1,1,1,1], 50, 80, 82, 83, 80, 100, 'water')

Caterpie   = Pokemon('Caterpie'  , 'bug', ['Missilspillo', 'Azione', 'Ragnatela', 'Coleomorso']   , [1,0,1,1], 5, 45, 30, 35, 45, 20, 'bug')
Metapod    = Pokemon('Metapod' , 'bug', ['Missilspillo', 'Rafforzatore', 'Ragnatela', 'Coleomorso'], [1,0,1,1], 25, 50, 20, 55,30, 25, 'bug' )
Butterfly  = Pokemon('Butterfly' , 'bug', ['Missilspillo', 'Azione', 'Ragnatela', 'Coleomorso']   , [1,0,1,1], 50, 60, 45, 50, 70, 80, 'flying' )

Weedle     = Pokemon('Weedle'  , 'bug', ['Missilspillo', 'Azione', 'Ragnatela', 'Coleomorso']   , [1,0,1,1], 5, 45, 30, 35, 45, 20, 'bug')
Kakuna     = Pokemon('Kakuna' , 'bug', ['Missilspillo', 'Rafforzatore', 'Ragnatela', 'Coleomorso'], [1,0,1,1], 25, 50, 20, 55,30, 25, 'bug' )
Beedril    = Pokemon('Beedril' , 'bug', ['Missilspillo', 'Azione', 'Ragnatela', 'Coleomorso']   , [1,0,1,1], 50, 60, 45, 50, 70, 80, 'flying' )

Pidgey     = Pokemon('Pidgey', 'flying', ['Azione', 'Attacco rapido', 'Raffica', 'Turbosabbia'] , [0,0,1,0],  5, 40, 45 , 40, 56, 35, 'normal')
Pidgeotto  = Pokemon('Pidgeotto', 'flying', ['Volo', 'Attacco rapido', 'Raffica', 'Alacciaio'] , [1,0,1,0],  25, 63, 60, 55, 71, 50, 'normal')
Pidgeot    = Pokemon('Pidgeot', 'flying', ['Volo', 'Aeroattacco', 'Aerosio', 'Alacciaio'] , [1,1,1,0],  50, 83, 80, 75, 91, 70, 'normal')

Rattata    = Pokemon('Rattata'   ,'normal', ['Azione', 'Colpocoda', 'Morso', 'Attacco rapido']   , [1,1,0,1], 10, 30, 56, 35, 72, 25, 'normal')
Raticate   = Pokemon('Raticate'  ,'normal', ['Gigaimpatto', 'Segugio', 'Sgranocchio', 'Extrarapido']   , [1,1,0,1], 50, 55, 81, 60, 97, 50, 'normal')

Spearow    = Pokemon('Spearow', 'flying', ['Azione', 'Attacco rapido', 'Raffica', 'Turbosabbia'] , [0,0,1,0],  5, 40, 60, 30, 70, 31, 'normal')
Fearow     = Pokemon('Fearow', 'flying', ['Volo', 'Aeroattacco', 'Aerosio', 'Alacciaio'] , [1,1,1,0],  50, 65, 90, 65, 100, 61, 'normal')

Ekans      = Pokemon('Ekans'   ,'poison', ['Azione', 'Colpocoda', 'Morso', 'Velenospina']   , [0,0,0,1], 10, 35, 60, 44, 55, 40, 'poison')
Arbok      = Pokemon('Arbok'  ,'poison', ['Sporcolancio', 'Tossina', 'Sgranocchio', 'Acido']   , [1,1,0,1], 50, 60, 85, 69, 80, 65, 'poison')

Pikachu    = Pokemon('Pikachu'   ,'electric', ['Azione', 'Colpocoda', 'Fulmine', 'Attacco rapido']   , [0,0,1,0], 10, 35, 55, 30, 90, 50, 'electric')
Raichu     = Pokemon('Raichu'  ,'electric', ['Tuono', 'Locomovolt', 'Codacciaio', 'Fulmine']   , [1,1,0,1], 50, 60, 90, 55, 100, 90, 'electric')


In [28]:
pokemonList = [Bulbasaur, Ivysaur, Venusaur,
              Charmander, Charmeleon, Charizard,
              Squirtle, Wartortle, Blastoise, Rattata, Raticate]

player, opponent = random.sample(pokemonList, 2)
Winner, Loser = player.startBattle(opponent, True, True)

--- BATTAGLIA POKEMON ---
 Raticate - 75HP
    printf("                        |.     .|\n");
    printf("                      `.  `._.' |,'Y'     _.......\n");
    printf("      +--------..  _\\\"'  \"\"\"\"\"\"'--.=-_ ,-'  ,.-- '     .\n");
    printf("       |  '\"\"`.  `.`-._           .-\" |   .'    (      |`\n");
    printf("       j       \\  |..'-- ,-----. ,.]..|  /       `.    L .\n");
    printf("  ____(___     |      _.' -  , `--..    | __.....-/-..__|L\n");
    printf(".'._______\"\"\"\"----  ,'   _____._    ` ,-':,...------\"\"\"\"i .\n");
    printf("         |\"\"\"\"-.  -'    '.     /`    ' -------.j__      | |\n");
    printf("     .,--------        / \\    j  L      `=..-\"\"----'    | |\n");
    printf("           ,-_,.-     j   L | |   .     `-..:-.__       | |\n");
    printf("        ,++-.  |      |   /-+-|   |       | `\"-._`._    | |\n");
    printf("      .+\"\" '- .'      L  j  | L   j       | L    `-.`.  F-|\n");
    printf("    ,'    .-) `,       \\_/     \

In [37]:
import copy
class trainer:
    def __init__(self, name, pokemons, currently_active):
        self.name = name
        if len(pokemons) > 6:
            print("Maximum 6 Pokemons allowed!")
        else:
            self.pokemons = pokemons
            self.pokemon_list = []
        for a in pokemons:
            self.pokemon_list.append(a.name)
        self.currently_active = self.pokemons[currently_active] 
        self.curr_active = self.pokemon_list[currently_active] 
    
def alive(pokemon): #check if the pokemon is alive
    if pokemon.HP>0:
        return True
    if pokemon.HP<=0:
        pokemon.HP=0
        return False
def battle(trainer1, trainer2): #battle between two trainers
    fight1, fight2 = True, True
    while fight1 and fight2:
        print ('Choose a pokemon:', trainer1.pokemons)
        pokemon1 = eval(input())
        while not alive(pokemon1):
            print (pokemon1.name, 'is out of batlle. Choose another pokemon')
            pokemon1 = eval(input())
        pokemon2 = random.choice(trainer2.pokemons)
        while not alive(pokemon2):
            pokemon2 = random.choice(trainer2.pokemons)
        print (trainer1.name, 'chose', pokemon1.name, '\n', trainer2.name, 'chose', pokemon2.name)
        while alive(pokemon1) and alive(pokemon2):
            Winner, Loser = pokemon1.startBattle(pokemon2, True, True)
        #print (trainer1.pokemon[0].name, trainer1.pokemon[0].hp) #here its returning the original hp
        #print (trainer2.pokemon[0].name, trainer2.pokemon[0].hp) #here its returning the current hp, as it should
        if not any(alive(pokemon) for pokemon in trainer1.pokemons):
            fight1 = False
        if not any(alive(pokemon) for pokemon in trainer2.pokemons):
            fight2 = False
    money = random.randint(100,5000)
    if not(fight2): delay_print(f"\nDai all'avversario ${money}.\n")
    else:
        delay_print(f"\nL'avversario ti da' ${money}.\n")
        return pokemon2, self
S1 = [Charizard, Squirtle]
S2 = [Rattata, Raticate]
S3 = [Weedle, Caterpie, Beedril, Butterfly]
S4 = [Pidgey, Pidgeotto, Pidgeot]
S5 = [Arbok, Fearow]
I = trainer('You', S1, 1) 
Genny = trainer('Gennaro Bullo', S2, 1)
Pigliamosche = trainer('Barucci Pigliamosche', S3, 1)
G_M = trainer('Giulia Mulattieri', S4, 1)
Popoto = trainer('Popoto', S5, 1)
battle(I, Genny)

Choose a pokemon: [<__main__.Pokemon object at 0x0000021DCBBD65E0>, <__main__.Pokemon object at 0x0000021DCBBD6700>]
Charizard
You chose Charizard 
 Gennaro Bullo chose Rattata
--- BATTAGLIA POKEMON ---
 Charizard - 80HP
                 .\"-,.__\
                 `.     `.  ,\
              .--'  .._,'\"-' `.\
             .    .'         `'\
             `.   /          ,'\
               `  '--.   ,-\"'\
                `\"`   |  \\\
                   -. \\, |\
                    `--Y.'      ___.\
                         \\     L._, \\\
               _.,        `.   <  <\\                _\
             ,' '           `, `.   | \\            ( `\
          ../, `.            `  |    .\\`.           \\ \\_\
         ,' ,..  .           _.,'    ||\\l            )  '\".\
        , ,'   \\           ,'.-.`-._,'  |           .  _._`.\
      ,' /      \\ \\        `' ' `--/   | \\          / /   ..\\\
    .'  /        \\ .         |\\__ - _ ,'` `        / /     `.`.\
    |  '         

AttributeError: 'int' object has no attribute 'HP'